## Bot is still working in Progress



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@markdown ##**Install library**
!pip install auto1111sdk

In [ ]:
#Create Enviroment for project

import shutil
import gc

import os
import sys
from IPython.display import display, HTML
import ipywidgets as widgets
import requests
from io import BytesIO

from auto1111sdk import civit_download, download_realesrgan, RealEsrganPipeline, StableDiffusionPipeline, EsrganPipeline
from PIL import Image
import torch



In [ ]:
def get_model():
  # Specify the path of the file in Google Drive
  drive_path = "/content/drive/MyDrive/AI/Models/dreamshaper.safetensors"

  # Specify the destination path in Colab
  model_path = 'models/dreamshaper.safetensors'  # You can change the filename or path if needed

  # Copy the file from Google Drive to Colab
  shutil.copy(drive_path, model_path)

def create_dir():
  # Create dir for images
  image_dir_path ='/content/images'
  model_dir_path ='/content/models'
  os.mkdir(image_dir_path)
  os.mkdir(model_dir_path)


def clean_gpu():
  # Delete unused variables
  del_vars = [var for var in globals() if isinstance(var, torch.Tensor)]
  for var in del_vars:
      del var

  # Empty the cache
  torch.cuda.empty_cache()

  # Run the garbage collector
  gc.collect()

# Download Model
create_dir()
# get_model()

Function for generation

In [ ]:
def generate_image(prompt, negative_prompt, num_images=1, height=768, width=512, steps=20,
                   output_path="/content/images/txt2img.png", cfg_scale=7.5, seed=-1, sampler_name='Euler'):


    # Create the pipeline
    print(model_path)
    pipe = StableDiffusionPipeline(model_path)

    # Generate the image
    output = pipe.generate_txt2img(num_images=num_images, cfg_scale=cfg_scale,
                                   sampler_name=sampler_name, seed=seed,
                                   prompt=prompt, height=height, width=width,
                                   negative_prompt=negative_prompt, steps=steps)

    # Construct the output path
    base_dir, filename = os.path.split(output_path)
    base_filename, extension = os.path.splitext(filename)
    counter = 0

    # Save each generated image with a unique filename
    for i in range(num_images):
        counter = 0
        output_path = os.path.join(base_dir, f"{base_filename}{counter}{extension}")

        # While a file already exists at the output path, increment the counter and construct a new output path
        while os.path.exists(output_path):
            counter += 1
            output_path = os.path.join(base_dir, f"{base_filename}{counter}{extension}")

        # Save the generated image
        output[i].save(output_path)

        # Display the generated image
        display(output[i])

    # Cleanup - delete the pipeline
    del pipe



Helper Function

In [ ]:
#Start up project
# model_path = 'dreamshaper.safetensors'
model_path = '/content/drive/MyDrive/AI/Models/dreamshaper.safetensors'
prompt = ''
negative_prompt = ''
num_images=1
height=768
width=512
steps=20
output_path="/content/images/txt2img.png"
cfg_scale=7.5
seed=-1
sampler_name='Euler'
easy_mode = True
generate_direct = False
model = ""

#Create the field
def create_input_field(description, placeholder):
    return widgets.Text(
        value='',
        placeholder=placeholder,
        description='',
        layout=widgets.Layout(width="500px")
    )

# Now for model path
def change_model(keyword) :
  global model_path
  if keyword.lower() == 'dream' :
    print("Loading Dream Model")
    model_path = "/content/drive/MyDrive/AI/Models/dreamshaper.safetensors"
  elif keyword.lower() == 'cartoon' :
    print("Loading Cartoon Model")
    model_path = "/content/drive/MyDrive/AI/Models/realcartoon3d.safetensors"
  else:
    print("Model Not Found, Use active model")

## COMMANDS
# Define the test_prompt function
def test_prompt():
    print("Using test prompt")
    prompt = "closeup portrait photo of beautiful 20 y.o man, 8k uhd, high quality, dramatic, cinematic"
    negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime), text, cropped, out of frame, worst quality, low quality, jpeg artifacts, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"
    return prompt, negative_prompt

def template_negative_prompt(keyword) :
  negative_prompt = ""
  if keyword.lower() == 'person':
    negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime), text, cropped, out of frame, worst quality, low quality, jpeg artifacts, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"
  if keyword.lower() == 'anime':
     negative_promp = ",(worst quality:2),(low quality:2),(normal quality:2),lowres,bad anatomy,bad hands,normal quality,((watermark)),5-funny-looking-fingers:2,"
  return negative_prompt

def command_function(command):
    print("COMMAND")
    generate_direct = False
    if command.lower().startswith('test'):
        generate_direct = True
        prompt, negative_prompt = test_prompt()
    elif command.lower().startswith('model '):
        model = command.split()[-1]
        print(model)
        change_model(model)
        generate_direct = True
        prompt, negative_prompt = test_prompt()
    else:
        prompt = "closeup portrait photo of beautiful 20 y.o man, 8k uhd, high quality, dramatic, cinematic"
        negative_prompt = ""
    return prompt, negative_prompt, generate_direct


In [ ]:
def bot():
    clean_gpu()
    prompt = ""
    negative_prompt = ""
    generate_direct = False
    if easy_mode:
      print("EASY MODE")
      # change_model('dir')
      prompt_input = create_input_field('', 'Enter the prompt:')
      display(prompt_input)
      negative_prompt_input = create_input_field('', 'Enter the negative prompt:')

      def on_enter_prompt(prompt_input, negative_prompt_input):
        nonlocal prompt
        nonlocal generate_direct
        prompt = prompt_input.value
        if prompt.lower().startswith('cmd '):
            command = ' '.join(prompt.split()[1:])
            prompt, negative_prompt, generate_direct = command_function(command)
            if generate_direct:
                generate_image(prompt, negative_prompt)
        if not generate_direct:
            display(negative_prompt_input)

      def on_enter_negative_prompt(negative_prompt_input):
          nonlocal prompt, negative_prompt
          negative_prompt = negative_prompt_input.value
          if negative_prompt.lower() in ['person', 'anime']:
              negative_prompt = template_negative_prompt(negative_prompt)
          generate_image(prompt, negative_prompt)


      prompt_input.on_submit(lambda _: on_enter_prompt(prompt_input, negative_prompt_input))
      negative_prompt_input.on_submit(lambda _: on_enter_negative_prompt(negative_prompt_input))


In [ ]:
# Run the bot
bot()